In [24]:
# import numpy as np
# import pandas as pd
# import os
#
#
# from afinn import Afinn
#
# from collections import Counter
# from sklearn.feature_extraction.text import CountVectorizer
# import matplotlib.pyplot as plt
# import networkx as nx

import spacy
from nltk import sent_tokenize, word_tokenize
import itertools
import en_core_web_sm
import operator

from nervaluate import Evaluator
from nltk.tokenize import sent_tokenize




In [25]:
nlp = en_core_web_sm.load()

In [26]:
def read_text(path):
    with open(path, encoding="utf-8") as f:
        text = f.read()
        text = text.replace('\r', ' ').replace('\n', ' ')\
            .replace("’", "'").replace("\"", "").replace("”", "").replace("“", "")
    return text

def spacy_NER(book, eval = False):
    """
    spacy_NER vrne seznam, v katerem so shranjene prepoznane identitete glede na posamezni stavek
    :param book: str
    :return: entity_dict (seznam slovarjev kot npr. [name, tag, start_pos, stop_pos, line_num. token_num])
    """

    # EVAL ---
    if eval == True:
        sentences, sentences_tag = conll_read()
        #print(len(sentences), "--------")
    else:
        sentences = sent_tokenize(book)
    sentences_NER = []
    sentences_NER_results = []
    # EVAL ---

    # 00 Pretvori knjigo v stavke:
    #sentences = sent_tokenize(book)
    entity_dict = []


    for line_num, line in enumerate(sentences):
        #print(line_num)

        # EVAL ---
        words_list = word_tokenize(line)
        tags_list = ["O"] * len(words_list)
        sentences_NER.append(words_list)
        # EVAL ---

        entities = nlp(line)
        for word in entities.ents:
            #print(word.text,word.label_, word.start, word.end)
            info_dict = {}
            info_dict["name"] = word.text
            info_dict["tag"] = word.label_
            info_dict["start_pos"] = word.start
            info_dict["stop_pos"] = word.end
            info_dict["line_num"] = line_num
            entity_dict.append(info_dict)

            # EVAL: ---
            names = word.text.replace("'s", "")
            names = names.split(" ")
            ff_flag = False
            if word.label_ == "PERSON":
                for name in names:
                    try:
                        if ff_flag == False:
                            idx = words_list.index(name)
                            tags_list[idx] = "B-PER"
                            ff_flag = True
                        else:
                            idx = words_list.index(name)
                            tags_list[idx] = "I-PER"
                    except:
                        print("Not found: ", name)
            # EVAL ---
        sentences_NER_results.append(tags_list)
    return entity_dict, sentences_NER, sentences_NER_results


def get_names_from_NER(entity_dict):
    """
    get_names_from_NER sprejme entity_dict in vrne urejen seznam terk ("ime", št_zaznano)
    :param entity_dict: dict (seznam dictov)
    :return: unique_names: list
    """
    unique_names = {}

    for entity in entity_dict:
        #print(entity)
        if entity["tag"] == "PERSON":
            if entity["name"] not in unique_names:
                unique_names[entity["name"]] = 1
            else:
                unique_names[entity["name"]] += 1
    unique_names = sorted(unique_names.items(), key=operator.itemgetter(1),reverse=True)

    return unique_names

def conll_read():
    with open("conll2003/valid.txt") as f:
        lines = f.readlines()
    sentences = []
    sentences_tag = []
    tmp = ""
    tmp_tag = ""
    tags = []
    for i in range(len(lines)):

        line = lines[i]
        line = line.split(" ")
        text_word = line[0]

        if text_word[0] != "-":
            if text_word == "\n":
                # print(tmp)
                # print(tmp_tag)
                # print("-------")
                sentences.append(tmp)
                tmp = ""
                #sentences_tag.append(tmp_tag)
                #tmp_tag = ""
                sentences_tag.append(tags)
                tags = []

            elif text_word == "(" or text_word == ")" or text_word == '"' or text_word == ':':
                pass
            else:
                if text_word[0] != "." and text_word[0] != "," and text_word[0] != ":" and text_word[0] != ";"\
                        and text_word[0] != "!" and text_word[0] != "?" and text_word[0] != ")" and text_word[0] != "'":   #or text_word[0] != "," or text_word[0] != "(" or text_word[0] != ")":
                    tmp = tmp + " " + text_word
                    line[3] = line[3].replace("\n", "")

                    if line[3] == "B-PER" or line[3] == "I-PER":
                        #line[3] = "B-PER"
                        #tmp_tag = tmp_tag + " " + line[3]
                        tags.append(line[3])
                    else:
                        line[3] = "O"
                        #tmp_tag = tmp_tag + " " + line[3]
                        tags.append(line[3])
                else:
                    tmp = tmp +  text_word
                    line[3] = "O"
                    tags.append(line[3])
    return sentences, sentences_tag



In [27]:
book = read_text('../../data/books/ASongOfIceAndFire/AGOT/chapters/1_Bran_1.txt')

entity_dict, sentences_NER, sentences_NER_results = spacy_NER(book, False)
unique_names = get_names_from_NER(entity_dict)

for (name, num) in unique_names:
    if True: #num > 1:
        print(name, num)



Jon 15
Theon Greyjoy 4
Greyjoy 4
Hullen 4
Stark 3
Jon Snow 3
Jory Cassel 2
Eddard Stark 1
Rhoynar 1
Robert 1
Bran 1
Harwin 1
Ser Rodrik's 1
Rickon 1
Snow 1
Jon thoughtfully 1
Desmond 1


In [22]:
# Evalvacija na datasetu Conll2003 (osredotočena na imena)

def eval_wrapper():
    entity_dict, sentences_NER, sentences_NER_results = spacy_NER("", eval= True)
    unique_names = get_names_from_NER(entity_dict)

    # Kontrola podatkov
    sentences_org, sentences_org_tag = conll_read()
    # for x in range(len(sentences_NER)):
    #     print("-----------", x)
    #     #print(len(sentences_org_tag[x]))
    #     print(len(sentences_NER[x]))
    #     print(len(sentences_NER_results[x]))

    #     #print(sentences_org[x])    # 22 26
    #     print(sentences_NER[x])
    #     print(sentences_org_tag[x])
    #     print(sentences_NER_results[x])
    
    true = sentences_org_tag
    pred = sentences_NER_results
    evaluator = Evaluator(true, pred, tags=['PER'], loader="list")

    results, results_by_tag = evaluator.evaluate()
    print(results)

In [29]:
## Evalvacija za I. poglavje GoT

from chapter__1annotated import chapter1_lst

# Kontrola:
# print(len(sentences_NER_results))
# print(len(chapter1_lst))

# for x in range(len(sentences_NER)):
#     print("-----------", x)
#     print(len(sentences_NER[x]))
#     print(len(sentences_NER_results[x]))
#     print(len(chapter1_lst[x]))
    

#     print(sentences_NER[x])
#     print(sentences_NER_results[x])
#     print(chapter1_lst[x])

true = chapter1_lst
pred = sentences_NER_results
evaluator = Evaluator(true, pred, tags=['PER'], loader="list")

results, results_by_tag = evaluator.evaluate()
print(results)

{'ent_type': {'correct': 44, 'incorrect': 0, 'partial': 0, 'missed': 120, 'spurious': 1, 'possible': 164, 'actual': 45, 'precision': 0.9777777777777777, 'recall': 0.2682926829268293, 'f1': 0.42105263157894746}, 'partial': {'correct': 41, 'incorrect': 0, 'partial': 3, 'missed': 120, 'spurious': 1, 'possible': 164, 'actual': 45, 'precision': 0.9444444444444444, 'recall': 0.25914634146341464, 'f1': 0.40669856459330145}, 'strict': {'correct': 41, 'incorrect': 3, 'partial': 0, 'missed': 120, 'spurious': 1, 'possible': 164, 'actual': 45, 'precision': 0.9111111111111111, 'recall': 0.25, 'f1': 0.39234449760765555}, 'exact': {'correct': 41, 'incorrect': 3, 'partial': 0, 'missed': 120, 'spurious': 1, 'possible': 164, 'actual': 45, 'precision': 0.9111111111111111, 'recall': 0.25, 'f1': 0.39234449760765555}}
